In [1]:
import pandas as pd
import numpy as np
import os

from glob import glob
from sklearn.metrics import accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score

In [2]:
source_dir = '../../zimp_orchestrator/orch/resources'
files = {}

for ds_path in glob(source_dir + '/*'):
    if not os.path.isdir(ds_path):
        continue
    ds_name = os.path.basename(ds_path)
    files[ds_name] = [os.path.join(ds_path, 'train.csv'), os.path.join(ds_path, 'test.csv')]

In [3]:
def calc_metrics(df_pred, metric_prefix=''):
    metrics = {}
    for metric_name, fun_score in [('accuracy', accuracy_score), ('balanced_accuracy', balanced_accuracy_score)]:
        metrics[metric_prefix + metric_name] = fun_score(df_pred['target'], df_pred['prediction'])

    for avg in ['micro', 'macro', 'weighted']:
        for metric_name, fun_score in [('f1', f1_score), ('precision', precision_score), ('recall', recall_score)]:
            metrics[metric_prefix + metric_name + '_' + avg] = fun_score(df_pred['target'], df_pred['prediction'],
                                                                         average=avg)

    metrics[metric_prefix + 'certainty_pos'] = df_pred[df_pred['target'] == df_pred['prediction']][
        'certainty'].mean()
    metrics[metric_prefix + 'certainty_neg'] = df_pred[df_pred['target'] != df_pred['prediction']][
        'certainty'].mean()
    metrics[metric_prefix + 'count_observations'] = df_pred.shape[0]
    
    return metrics

In [4]:
all_metrics = {}
for dataset in files:
    df_train = pd.read_csv(files[dataset][0])
    pred_val = df_train.target.value_counts().index[0]
    certainty = df_train.target.value_counts().max()/df_train.shape[0]
    metrics = calc_metrics(pd.DataFrame({'target': df_train.target, 'prediction': pred_val, 'certainty': certainty}), 'train_')
    df_test = pd.read_csv(files[dataset][1])
    metrics = {**metrics, **calc_metrics(pd.DataFrame({'target': df_test.target, 'prediction': pred_val, 'certainty': certainty}), 'test_')}
    all_metrics[dataset] = metrics
    
df_metrics = pd.DataFrame(all_metrics).T
df_metrics.to_csv('dummy_baseline.csv')
df_metrics

E:\Python\miniconda3\envs\zimp\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\Python\miniconda3\envs\zimp\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\Python\miniconda3\envs\zimp\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
E:\Python\miniconda3\envs\zimp\lib\site-packages\sklearn\metrics\_classifica

,train_accuracy,train_balanced_accuracy,train_f1_micro,train_precision_micro,train_recall_micro,train_f1_macro,train_precision_macro,train_recall_macro,train_f1_weighted,train_precision_weighted,...,test_recall_micro,test_f1_macro,test_precision_macro,test_recall_macro,test_f1_weighted,test_precision_weighted,test_recall_weighted,test_certainty_pos,test_certainty_neg,test_count_observations
10K-GNAD,0.163332,0.111111,0.163332,0.163332,0.163332,0.031200,0.018148,0.111111,0.045863,0.026677,...,0.163424,0.031215,0.018158,0.111111,0.045912,0.026707,0.163424,0.163332,0.163332,1028.0
DBP-14,0.071429,0.071429,0.071429,0.071429,0.071429,0.009524,0.005102,0.071429,0.009524,0.005102,...,0.071429,0.009524,0.005102,0.071429,0.009524,0.005102,0.071429,0.071429,0.071429,70000.0
GERMEVAL-2018,0.663007,0.500000,0.663007,0.663007,0.663007,0.398679,0.331503,0.500000,0.528654,0.439578,...,0.661566,0.398158,0.330783,0.500000,0.526815,0.437669,0.661566,0.663007,0.663007,3398.0
GERMEVAL-2020,0.410295,0.200000,0.410295,0.410295,0.410295,0.116371,0.082059,0.200000,0.238733,0.168342,...,0.411005,0.116514,0.082201,0.200000,0.239439,0.168925,0.411005,0.410295,0.410295,20900.0
TREC-6,0.229274,0.166667,0.229274,0.229274,0.229274,0.062170,0.038212,0.166667,0.085524,0.052566,...,0.188000,0.052750,0.031333,0.166667,0.059502,0.035344,0.188000,0.229274,0.229274,500.0
YELP-5,0.200000,0.200000,0.200000,0.200000,0.200000,0.066667,0.040000,0.200000,0.066667,0.040000,...,0.200000,0.066667,0.040000,0.200000,0.066667,0.040000,0.200000,0.200000,0.200000,50000.0
